# Optimizing Bids for Keywords

Google keyword planner provides forecasts like the expected impressions, clicks, and conversions for different keywords depending on the daily budget **specified by the user**. This forecasts are based on **historical data, usage patterns for users and trends** exhibited across google for these keywords. Also, it provides the minimum and maximum **top of page bids** required for a given keyword **i.e.** bidding **10$** for a keyword with maximum top of page bid of **8$** **does not** lead to extra clicks. We used this data to determine how much to bid for each keyword that lead to **maximum** number of clicks given the specified budget. Hence, maximizing budget usage.

By visualizing the forecasts from the keyword planner, we notice that keywords exhibit exponential growth and saturate at a certain daily budget which leads to the conclusion that after a certain point, bidding more does **not** lead to more clicks. By automating the process of setting the bids for each keyword, we achieve maximum utilization of the budget. Additionally, we can use this method for filtering out keywords that have bad conversion and suggesting ones that result in a high number of clicks.

![Google Keyword Planner](assets/kw_planner.png "Google Keyword Planner")

## Method

We define the relationship between the estimated number of clicks $clicks_k$, and the allocated daily budget $budget_k$ for the $k^{th}$ keyword using the function $f_k$ by interpolating the data from the graph above.

$$clicks_k = f_k(budget_k)$$

We define our problem as follows:

$$\underset{budget_k}{\text{argmax}}\; \quad {clicks_{total}} = \sum_{k=0}^{K}{f_k(budget_k)}$$

$$\text{subject to:} \quad \sum_{k=0}^{K} budget_k \leq budget_{total} \text{  ,}$$

$$\quad budget_k \leq budget_{max}^k$$

where:
$$\text{K: Number of keywords}$$
$$f_k \text{: Interpolated function for the $k^{th}$ keyword}$$
$$budget_{total} \text{: Total daily budget of all keywords set by the user}$$
$$budget_{max}^k \text{: Budget at saturation point for the $k^{th}$ keyword}$$

The objective function is subject to the following constraints:
1. Sum of allocated budgets must not exceed the total budget specified by the user. 
2. Allocated budget for each keyword must not exceed the saturation point from the graph.

We picked 4 keywords in the Weleda campaign, and got the forecasted number of clicks at different budgets **from 1-6-2023 to 30-6-2023** from the keyword planner. The table below shows the saturation point for each keyword. From the table, we can deduce that the maximum number of clicks for these keywords will not exceed **245 clicks** regardless of the budget.

| Keyword | Max Budget | Max Clicks |
|---------|:-----------:|-------------|
| weleda hair tonic | 17 | 44 |
| weleda sunscreen | 2.08 | 15 |
| weleda solskydd | 16 | 76 |
| skinfood weleda | 10 | 110 |
| **Total** | **45.08** | **245**



We fit a model between the clicks and daily budget for each keyword, and set a total daily budget. Then, we maximized the objective function subject to the above constraints. The maximized function sets the optimal bid for each keyword that **maximizes $clicks_{total}$** given the daily budget. We tested the model with different budgets **[200, 40, 10]** and equally distributed it to each keyword. Then, we calculated $clicks_{total}$ before and after maximization. The optimal bids lead to the budget allocation that results in the highest number of clicks. Additionally, it accounts for saturation; it lowers the initial budget if it does not result in more clicks.

at **Budget = 200**:

| Keyword | Initial Bid | Initial Clicks | Optimal Bid | Optimal Clicks
|-------------|:--------------:|---------------|-------| ---------------
| weleda hair tonic | 50 | 44 | 17 | 44
| weleda sunscreen | 50 | 15 | 2.08 | 15
| weleda solskydd | 50 | 76 | 16 | 76
| skinfood weleda | 50 | 110 | 10 | 110
| **Total** | **200** | **245** | **45.08** | **245**


at **Budget = 40**:

| Keyword | Initial Bid | Initial Clicks | Optimal Bid | Optimal Clicks
|-------------|:--------------:|---------------|-------| ---------------
| weleda hair tonic | 10 | 36 | 12 | 38
| weleda sunscreen | 10 | 15 | 2.08 | 15
| weleda solskydd | 10 | 66 | 15.92 | 76
| skinfood weleda | 10 | 110 | 10 | 110
| **Total** | **40** | **227** | **40** | **239**


at **Budget = 10**:

| Keyword | Initial Bid | Initial Clicks | Optimal Bid | Optimal Clicks
|-------------|:--------------:|---------------|-------| ---------------
| weleda hair tonic | 2.5 | 19 | 1.56 | 15
| weleda sunscreen | 2.5 | 15 | 1.05 | 13
| weleda solskydd | 2.5 | 35 | 2.29 | 34
| skinfood weleda | 2.5 | 52 | 5.1 | 81
| **Total** | **10** | **122** | **10** | **143**


## Code

In [43]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, shgo
from scipy.interpolate import interp1d
import os

In [17]:
kw_lst=["weleda_hair_tonic.csv","weleda_sunscreen.csv","weleda_solskydd.csv","skinfood_weleda.csv"]
data_dir = "data-forecast-1-6_30-6"
funcs=[]
for i in range(len(kw_lst)):
    data = pd.read_csv(os.path.join(data_dir,kw_lst[i]))
    data["daily_budget"] = data["daily_budget"].astype(float)
    data["clicks"] = data["clicks"].astype(float)
    funcs.append(interp1d(data["daily_budget"],data["clicks"],bounds_error=False, fill_value=(0,data["clicks"].values[-1])))

In [52]:
# clicks function
# maximizing f(x) is equivalent to minimizing -f(x)
def objective(x):
  return -sum([sum(func([xi])) for xi,func in zip(x,funcs)])


# NOT USED IN ANY OF THE CONSTRAINTS
# JUST PRINTED
MAX_TOTAL_CLICKS = np.array([44,15,76,110])

# EXTRACTED FROM THE KEYWORD PLANNER
# USED IN CONSTRAINTS
MAX_DAILY_BUDGET = np.array([17,2.08,16,10])

# SET BY THE USER
# USED IN CONSTRAINTS
DAILY_BUDGET = 10


print(f"(Keyword, Max Daily Budget, Max Total Clicks): {list(zip(kw_lst,MAX_DAILY_BUDGET, MAX_TOTAL_CLICKS))}")
print(f"Max Daily Budget = {sum(MAX_DAILY_BUDGET)}, Max Total Clicks = {sum(MAX_TOTAL_CLICKS)}")
print()
  
constraints = [
  {'type':'ineq','fun': lambda x: np.sum(MAX_DAILY_BUDGET) - np.sum(x)},
  {'type':'ineq','fun': lambda x: DAILY_BUDGET - np.sum(x)}
]

bounds = np.zeros((MAX_DAILY_BUDGET.shape[0],2))
bounds[:,1] = MAX_DAILY_BUDGET

# GLOBAL MINIMIZER THAT SEARCHES PARAMETER SPACE FOR LOCAL MINIMAS AND SETS THE GLOBAL MINIMUM ACCORDINGLY
# DOES NOT REQUIRE AN INITIALIZATION
min = shgo(objective, constraints=constraints, bounds=bounds, options={"disp":True} ,iters=5, sampling_method='sobol')

# ILLUSTRATE THE DIFFERENCE BETWEEN EVENLY DISTRIBUTING THE BUDGET AND OPTIMIZING
EVEN_BIDS = np.array([DAILY_BUDGET/len(kw_lst)]*len(kw_lst))
print(f"Initial Bids = {EVEN_BIDS.round(2)}, Initial Budget = {DAILY_BUDGET:.2f}, Clicks = {round(-objective(EVEN_BIDS),0)}, Avg_CPC = {DAILY_BUDGET*30.4/-objective(EVEN_BIDS):.2f}")
print(f"Optimal Bids = {min['x'].round(2)}, New Budget = {min['x'].sum():.2f}, Clicks = {round(-min['fun'],0)}, Avg_CPC = {sum(min['x'])*30.4/-min['fun']:.2f}")

(Keyword, Max Total Clicks, Max Daily Budget): [('weleda_hair_tonic.csv', 17.0, 44), ('weleda_sunscreen.csv', 2.08, 15), ('weleda_solskydd.csv', 16.0, 76), ('skinfood_weleda.csv', 10.0, 110)]
Max Total Clicks = 245, Max Daily Budget = 45.08

Initial Bids = [2.5 2.5 2.5 2.5], Initial Budget = 10.00, Clicks = 122.0, Avg_CPC = 2.49
Optimal Bids = [1.56 1.05 2.29 5.1 ], New Budget = 10.00, Clicks = 143.0, Avg_CPC = 2.13
